In [1]:
cd /home

/home


In this notebook, we will try to create a function to read a a parquet file (or multiple ones) and filter by IDs.

In [2]:
import pandas as pd
import numpy as np
from os.path import join
import polars as pl
import os

In [3]:
pwd

'/home'

In [4]:
local_path = './data/raw_data/alcock/'

Read the metadata and save it into one parquet.

In [10]:
metadata = pd.read_csv(join(local_path, 'metadata.csv'))

In [11]:
path = (local_path + 'LCs/'+ metadata.Path).to_list()
IDs = metadata.ID.to_list()
metadata = metadata.assign(newID=metadata.index.values)

In [12]:
path_parquets = join(local_path, 'parquets/')
if not os.path.exists(path_parquets):
    os.mkdir(path_parquets)

In [13]:
dfs = []
for cont, (file, id_) in enumerate(zip(path, IDs)):
    # Read the file
    df = pd.read_csv(file, engine='c', na_filter=False)
    df['newID'] = metadata.newID.iloc[cont]*np.ones(df.shape[0]).astype(np.int64)
    dfs.append(df)
dfs = pd.concat(dfs) 
dfs = dfs.set_index('newID')

In [14]:
dfs

,mjd,mag,err
newID,,,
0,48882.58984,-9.515,0.002
0,48884.63672,-9.482,0.002
0,48885.60547,-9.500,0.001
0,48887.66797,-9.487,0.003
0,48888.72266,-9.378,0.006
...,...,...,...
21443,51304.37891,-4.940,0.078
21443,51308.37109,-5.072,0.064
21443,51315.36719,-4.926,0.090


Save every 1000 lightcurves


In [15]:
dfs.iloc[100000:101000]

,mjd,mag,err
newID,,,
144,48892.75781,-6.883,0.020
144,48894.72656,-6.888,0.031
144,48895.75391,-6.882,0.015
144,48896.75781,-6.668,0.016
144,48906.57422,-6.611,0.035
...,...,...,...
145,50276.75391,-5.500,0.038
145,50278.69922,-5.543,0.050
145,50296.75391,-5.613,0.071


In [16]:
nsamples = 1000
for batch, begin in enumerate(np.arange(0, dfs.shape[0], nsamples)):
    df_sel = dfs.iloc[begin:begin+nsamples]    
    n = str(batch).rjust(3, '0')
    df_sel.to_parquet(os.path.join(path_parquets, 
                                   'shard_'+n+'.parquet'))

In [17]:
metadata['Label'] = metadata['Class'].cat.codes

AttributeError: Can only use .cat accessor with a 'category' dtype

In [ ]:
metadata.to_parquet(join(local_path, 'new_metadata.parquet'), index=False)

In [ ]:
import polars as pl

In [ ]:
path_parquets = './data/raw_data/alcock/parquets/*'
# path_parquets = [os.path.join(root, x) for x in os.listdir(root)]
paths = os.path.join(path_parquets)
scan = pl.scan_parquet(paths)

In [ ]:
# scan.collect(streaming=False)['newID'].unique()

In [ ]:
import pandas as pd

In [ ]:
root = './data/raw_data/alcock/parquets/'
path_parquets =[os.path.join(root, x) for x in os.listdir(root)]

dfs = []
for file in path_parquets:
    df = pd.read_parquet(file)
    dfs.append(df)

In [ ]:
pd.concat(dfs).reset_index()['newID'].unique().shape

In [ ]:
pd.concat(dfs).shape